 Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 446.0 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using groq's `llama-3.2-11b-text-preview`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"]=userdata.get("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.2-11b-text-preview")


In [ ]:
llm.invoke('hi how are you')

AIMessage(content="I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or topics you'd like to discuss. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 39, 'total_tokens': 95, 'completion_time': 0.074666667, 'prompt_time': 0.007821141, 'queue_time': 0.00774383, 'total_time': 0.082487808}, 'model_name': 'llama-3.2-11b-text-preview', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2299003-06f4-4494-b2ad-fe418a3d1f74-0')

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    llm=llm,
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    llm=llm,
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "

                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer.

Final Answer:

**Content Plan Document: "Unlocking the Power of Artificial Intelligence"**

**I. Executive Summary**

This content plan aims to educate readers on the latest trends, key players, and noteworthy news in Artificial Intelligence (AI). By understanding the opportunities and challenges posed by AI, readers will be equipped to make informed decisions about its applications and adoption. This document will serve as a comprehensive guide for a blog article on AI, covering its history, types, applications, 

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Unlocking the Power of Artificial Intelligence

## I. Introduction

Artificial Intelligence (AI) has revolutionized the way we live and work, transforming industries and shaping the future of humanity. But what exactly is AI, and how has it evolved over time? In this article, we'll delve into the world of AI, exploring its history, types, applications, and future prospects. As we navigate the complexities of AI, it's essential to understand its various forms and how they impact our daily lives.

AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. The concept of AI dates back to the mid-20th century, when computer scientists began exploring ways to create machines that could think and learn like humans. From simple rule-based systems to complex neural networks, AI has come a long way in a relatively short period.

## II. Types of Artificial Intelligence

There are several types of AI, each with its own unique characteristics and applications.

### Narrow or Weak AI

Narrow or weak AI is designed to perform a specific task, such as image recognition, natural language processing, or playing chess. These systems are trained on large datasets and can learn to perform their tasks with high accuracy. For instance, virtual assistants like Siri and Alexa use narrow AI to understand and respond to voice commands.

### General or Strong AI

General or strong AI, on the other hand, refers to a system that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks. This type of AI is still in its infancy, but it has the potential to revolutionize the way we live and work. General AI can learn from experience, adapt to new situations, and make decisions based on complex data.

### Superintelligence

Superintelligence refers to an AI system that significantly surpasses human intelligence, potentially leading to exponential growth in technological advancements. While the concept of superintelligence is still theoretical, it has sparked intense debate and speculation about its potential implications. Some experts believe that superintelligence could lead to significant breakthroughs in fields like medicine and energy, while others worry about its potential risks.

## III. Applications of Artificial Intelligence

AI has a wide range of applications, from healthcare and finance to transportation and education.

### Machine Learning

Machine learning is a type of AI that enables systems to learn from data and improve their performance over time. This technology has been widely adopted in industries such as healthcare, finance, and marketing. For example, Google's AlphaGo AI system used machine learning to defeat a human world champion in the game of Go.

### Natural Language Processing

Natural language processing (NLP) is a type of AI that enables systems to understand, interpret, and generate human language. NLP has been used in applications such as chatbots, virtual assistants, and language translation software. For instance, Amazon's Alexa uses NLP to understand voice commands and respond accordingly.

### Computer Vision

Computer vision is a type of AI that enables systems to interpret and understand visual data from images and videos. This technology has been used in applications such as facial recognition, object detection, and self-driving cars. For example, Google's self-driving cars use computer vision to navigate roads and avoid obstacles.

### Expert Systems

Expert systems are a type of AI that mimic the decision-making abilities of a human expert in a particular field. These systems use a combination of rules, knowledge, and experience to provide accurate and informed decisions. For instance, medical expert systems can help doctors diagnose diseases and recommend treatments.

## IV. Key Players in the Artificial Intelligence Market

Several companies are at the forefront of the AI revolution, driving innovation and development in the field.

### Google

Google has been a leader in AI research and development, with a wide range of applications in areas such as machine learning, natural language processing, and computer vision. Google's AI-powered services, such as Google Assistant and Google Photos, have revolutionized the way we interact with technology.

### Microsoft

Microsoft has also been a major player in the AI market, with a focus on developing AI solutions for businesses and organizations. Microsoft's Azure Machine Learning platform provides businesses with a cloud-based platform to build, deploy, and manage AI models.

### Amazon

Amazon has been at the forefront of AI-powered e-commerce, using machine learning and natural language processing to enhance customer experience and improve operational efficiency. Amazon's Alexa virtual assistant has become a household name, and its AI-powered services have transformed the way we shop online.

### IBM

IBM has a long history of AI research and development, with a focus on areas such as expert systems, natural language processing, and computer vision. IBM's Watson AI platform provides businesses with a cloud-based platform to build, deploy, and manage AI models.

## V. Noteworthy News and Trends in Artificial Intelligence

AI has been making headlines in recent years, with breakthroughs and advancements in areas such as machine learning, natural language processing, and computer vision.

### Recent Breakthroughs in AI Research

Researchers have made significant progress in areas such as deep learning, transfer learning, and reinforcement learning, leading to improved performance and accuracy in AI systems. For example, researchers have made significant breakthroughs in areas such as image recognition and natural language processing.

### Emerging Applications of AI

AI is being used in a wide range of emerging applications, from healthcare and finance to transportation and education. For instance, AI-powered chatbots are being used in customer service, while AI-powered robots are being used in manufacturing and logistics.

### Challenges and Limitations of AI

While AI has the potential to revolutionize many industries and aspects of our lives, it also poses significant challenges and limitations, including bias, security, and ethics concerns. For example, AI systems can perpetuate biases and stereotypes, while AI-powered robots can pose safety risks.

## VI. Future Prospects of Artificial Intelligence

The future of AI is vast and exciting, with potential applications in areas such as healthcare, finance, transportation, and education.

### Potential Impact of AI on Various Industries

AI has the potential to transform numerous industries, from healthcare and finance to transportation and education. For instance, AI-powered diagnostic tools can help doctors diagnose diseases more accurately, while AI-powered robots can improve manufacturing and logistics.

### Expected Growth and Adoption of AI

The adoption of AI is expected to continue growing in the coming years, with more businesses and organizations embracing AI solutions. For example, AI-powered chatbots are becoming increasingly popular in customer service, while AI-powered robots are being used in manufacturing and logistics.

### Ethical Considerations and Concerns

As AI becomes increasingly prevalent, it's essential to address ethical concerns and ensure that AI is developed and used responsibly. For instance, AI systems should be designed to prevent bias and ensure fairness, while AI-powered robots should be designed to ensure safety and security.

## VII. Conclusion

In conclusion, AI has the potential to revolutionize many industries and aspects of our lives, but it also poses significant challenges and limitations. As we move forward, it's essential to address these concerns and ensure that AI is developed and used responsibly.

## VIII. Call to Action

We invite readers to explore AI further, learning more about its applications, benefits, and potential risks. Join the conversation and share your thoughts and opinions on AI.

### Resources

* [Google AI Blog](https://ai.googleblog.com/)
* [Microsoft AI Blog](https://blogs.microsoft.com/ai/)
* [Amazon AI Blog](https://aws.amazon.com/blogs/ai/)
* [IBM AI Blog](https://www.ibm.com/blogs/ai/)

### Related Articles

* [The Future of Artificial Intelligence](https://www.example.com/article1)
* [The Benefits and Risks of Artificial Intelligence](https://www.example.com/article2)
* [How to Get Started with Artificial Intelligence](https://www.example.com/article3)

### Tags

* Artificial Intelligence
* AI
* Machine Learning
* Natural Language Processing
* Computer Vision
* Expert Systems
* Superintelligence
* AI Trends
* AI Applications

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "ai for bigineer"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on ai for bigineer.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:

**Comprehensive Content Plan Document: AI for Beginners**

**I. Audience Analysis**

* **Demographics:** Beginners interested in AI, including students, professionals, and enthusiasts
* **Interests:** Understanding AI concepts, learning about AI applications, staying updated on AI trends
* **Pain Points:**
 + Difficulty in understanding AI basics
 + Limited knowledge about AI tools and platforms
 + Struggling to apply AI concepts in real-world scenarios
* **Psychographics:** Curious, eager to learn, and pass

In [ ]:
Markdown(result)

# AI for Beginners: Unlocking the Power of Artificial Intelligence

## Introduction

Welcome to the world of artificial intelligence (AI), where machines learn, adapt, and solve complex problems. As a beginner, you may be wondering where to start and what AI has to offer. In this comprehensive guide, we'll introduce you to the basics of AI, its applications, and the tools you need to get started.

Artificial intelligence has come a long way since its inception in the 1950s. From simple algorithms to complex neural networks, AI has revolutionized various industries, including healthcare, finance, education, and marketing. With the rise of machine learning and deep learning, AI has become more accessible and powerful, enabling businesses and individuals to harness its potential.

As a beginner, it's essential to understand the importance of AI in today's digital landscape. AI is not just a buzzword; it's a technology that can help you solve real-world problems, improve processes, and drive innovation. Whether you're a student, professional, or enthusiast, this guide will provide you with a solid foundation in AI and help you unlock its power.

## Key Points

### Machine Learning (ML) Basics

Machine learning is a subset of AI that enables machines to learn from data and improve their performance over time. There are three primary types of machine learning:

* **Supervised Learning:** In this type of learning, the algorithm is trained on labeled data to learn the relationship between inputs and outputs.
* **Unsupervised Learning:** In this type of learning, the algorithm is trained on unlabeled data to discover patterns and relationships.
* **Reinforcement Learning:** In this type of learning, the algorithm learns by interacting with an environment and receiving rewards or penalties.

Some popular machine learning algorithms include:

* **Decision Trees:** A decision tree is a tree-like model that splits data into subsets based on features and their values.
* **Random Forests:** A random forest is an ensemble learning method that combines multiple decision trees to improve accuracy.
* **Neural Networks:** A neural network is a type of machine learning model inspired by the structure and function of the human brain.

### Deep Learning (DL) Fundamentals

Deep learning is a subset of machine learning that uses neural networks with multiple layers to learn complex patterns in data. There are two primary types of deep learning:

* **Convolutional Neural Networks (CNNs):** CNNs are used for image and video recognition tasks, such as image classification, object detection, and image segmentation.
* **Recurrent Neural Networks (RNNs):** RNNs are used for sequential data tasks, such as speech recognition, natural language processing, and time series forecasting.

Some popular deep learning applications include:

* **Image Recognition:** AI can recognize objects, scenes, and activities in images and videos.
* **Speech Recognition:** AI can recognize and transcribe spoken language.
* **Natural Language Processing (NLP):** AI can analyze and generate human language.

### AI Tools and Platforms

There are numerous AI tools and platforms available, each with its strengths and weaknesses. Some popular AI tools and platforms include:

* **TensorFlow:** TensorFlow is an open-source machine learning framework developed by Google.
* **PyTorch:** PyTorch is an open-source machine learning framework developed by Facebook.
* **Keras:** Keras is a high-level neural networks API that can run on top of TensorFlow, PyTorch, or Theano.

When choosing an AI tool or platform, consider the following factors:

* **Ease of use:** Look for tools with intuitive interfaces and minimal setup requirements.
* **Scalability:** Choose tools that can handle large datasets and complex tasks.
* **Community support:** Select tools with active communities and extensive documentation.

### AI Applications in Real-World Scenarios

AI has numerous applications in real-world scenarios, including:

* **Healthcare:** AI can help diagnose diseases, develop personalized treatment plans, and improve patient care.
* **Finance:** AI can help detect fraud, manage portfolios, and assess risk.
* **Education:** AI can help develop adaptive learning systems, grade assignments, and analyze student performance.

Some common AI challenges and solutions include:

* **Overfitting and Underfitting:** Regularization techniques, such as dropout and early stopping, can help prevent overfitting and underfitting.
* **Regularization Techniques:** Techniques, such as L1 and L2 regularization, can help prevent overfitting.
* **Hyperparameter Tuning:** Techniques, such as grid search and random search, can help tune hyperparameters.

## Call to Action

Now that you've learned the basics of AI, it's time to start exploring and practicing. Here are some resources to help you get started:

* **Recommended Books:** "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville, and "Artificial Intelligence: A Modern Approach" by Stuart Russell and Peter Norvig.
* **Recommended Online Courses:** Coursera's "Artificial Intelligence Specialization" and edX's "Introduction to Artificial Intelligence".
* **Recommended Blogs and Websites:** Towards Data Science, KDnuggets, and AI Impacts.

Join the AI community and start exploring the possibilities of artificial intelligence. Share your experiences, tips, and projects with us, and let's unlock the power of AI together!

## Additional Resources

* **Recommended Books:**
 + "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville
 + "Artificial Intelligence: A Modern Approach" by Stuart Russell and Peter Norvig
* **Recommended Online Courses:**
 + Coursera: "Artificial Intelligence Specialization"
 + edX: "Introduction to Artificial Intelligence"
* **Recommended Blogs and Websites:**
 + Towards Data Science
 + KDnuggets
 + AI Impacts

## Conclusion

Artificial intelligence has come a long way, and its applications are vast and varied. As a beginner, you've taken the first step towards understanding the basics of AI and exploring its potential. Remember to practice, experiment, and learn from others to unlock the power of AI. Join the AI community, share your experiences, and let's shape the future of artificial intelligence together!

Note: This post has been reviewed for grammatical errors and alignment with the brand's voice.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).